###### 2020-11-22 일요일
###### 2020-11-23 월요일

## K-Fold 적용하지 않음

# 01_텍스트데이터토큰화 & 토큰사전저장
 - 음절단위 토큰화 진행 과정에서 많은 시간이 소요된다
 - 그래서 의미없는 시간 손실을 줄이기 위해 음절단위 토큰화를 진행하고 음절단위 토큰사전을 json파일로 저장하고자 한다.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import re
import json
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. 데이터 불러오기

In [3]:
d1             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data1.csv')
d2             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data2.csv')
d3             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data3.csv', encoding='cp949')
d4             = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data4.xlsx')
d5             = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data5.csv')
d6             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data6.csv')

In [5]:

# 모든데이터 하나의 데이터프레임으로 합치기
df_list = [d1, d2, d3, d4, d5, d6]

all_df = d1
for df in df_list[1:]:
    all_df = pd.concat([all_df, df])

In [6]:
# 필요없는 열 제거
all_df.drop(['Unnamed: 0', '출처'], axis=1, inplace=True)

# 라벨이 NaN인거 제거
index = all_df['악플여부'].isna()
raw_df = all_df[~index]

# 인덱스 리셋
raw_df = raw_df.reset_index()
raw_df.drop(['index'], inplace=True, axis=1)

In [8]:
raw_df.to_csv('3. 우리가 직접 0, 1, 2로 라벨링한 데이터셋.csv')

In [ ]:
# 고정 시드값 지정
seed = 123

# 댓글 길이 지정
comment_len = 400

##### 악플여부 라벨 1인 경우와 라벨 2인 경우를 하나로 합친다
 - 라벨이 1인 경우는 악플로 분류하기가 애매한 경우이나 대부분 사람들이 불쾌할만한 단어가 포함되어 있기 때문에 라벨1과 라벨2인 경우를 라벨1로 통합한다

In [ ]:
index = copy_df['악플여부'] == 2
copy_df.loc[index, '악플여부'] = 1

In [ ]:
copy_df['악플여부'] = copy_df['악플여부'].apply(int)
copy_df['댓글'] = copy_df['댓글'].apply(str)

In [ ]:
copy_df['악플여부'].value_counts()

0    69933
1    40591
Name: 악플여부, dtype: int64

## 2. 토큰화 & 저장
 - Train set, Test set을 나눈다
 - Train set을 기준으로 음절 토큰 사전을 만든다
 - Train set / Test set / 음절토큰사전을 저장한다
    - 이렇게 저장하는 이유는? : 매번 음절토큰사전을 만드는데 시간이 많이 소요되기 때문에 한번 만들어놓고 재사용하여 시간을 절약!


In [ ]:
def tokenization_save(df):

    feature = df['댓글']
    label = df['악플여부']

    X_train, X_test, y_train, y_test = train_test_split(feature, label, 
                                                    test_size=0.2,
                                                    random_state=seed)
    
    X_train_token_list = list(''.join(X_train))
    
    tokenizer = Tokenizer(oov_token = 'OOV')
    tokenizer.fit_on_texts(X_train_token_list)
    
    word_index_vocab = tokenizer.word_index

    return X_train, X_test, y_train, y_test, word_index_vocab


In [ ]:
def train_test_save(X_train, X_test, y_train, y_test) :
    save_df_list = [X_train, X_test, y_train, y_test]
    file_names = ['X_train', 'X_test', 'y_train', 'y_test']
    for idx, set in enumerate(save_df_list):
        set.to_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/' + file_names[idx] + '.csv', encoding='UTF-8-SIG')


def save_word_index(word_index_vocab) :
    json_file = json.dumps(word_index_vocab, ensure_ascii=False)
    f = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'w')
    f.write(json_file)
    f.close()

In [ ]:
# 토큰화
start = time.time()

X_train, X_test, y_train, y_test, word_index_vocab = tokenization_save(copy_df)

print("걸린시간 :", time.time() - start)

걸린시간 : 13.488031148910522


In [ ]:
# Train / Test 저장
train_test_save(X_train, X_test, y_train, y_test)

# 음절단위토큰 저장
save_word_index(word_index_vocab)